# Browser Automation Homework ROUND 2
Due 7-16<br>
Completed by: **Cassidy Jensen**

We're going to the United States trademark data base and going to collect all of Nike (or any company's) active trademarks.

We only want the live trademarks, and we want the `serial` number and the link to an image of each trademark. Feel free to collect the `wordmark` if you like.


Note: if you get asked if you're a bot, just complete the challenges manually.

In [97]:
import os
import random
import time

from playwright.async_api import async_playwright, expect, Keyboard

In [98]:
os.makedirs('data/', exist_ok=True)

In [99]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'

In [100]:
async def open_browser(headless=False, user_agent=user_agent):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.chromium.launch(headless=headless)

    # set a user agent
    context = await browser.new_context(user_agent=user_agent)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [101]:
driver, page = await open_browser()

python(11158) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [102]:
url = 'https://tmsearch.uspto.gov/search/'
await page.goto(url)

<Response url='https://tmsearch.uspto.gov/search/' request=<Request url='https://tmsearch.uspto.gov/search/' method='GET'>>

Only search by the "Owner" of each trademark

In [103]:
#xpath_search_options = '//*[@id="mat-select-2"]'
xpath_search_options = '//mat-select[@aria-label="Search refinment"]'
filter = page.locator(xpath_search_options)

In [104]:
#page.get_by_placeholder("Search for words in the owners of a mark")

In [105]:
await filter.click()

In [106]:
#xpath_option = '//[@id="mat-option-11"]'
xpath_option = '//mat-option[@id="mat-option-11"]'

In [107]:
await page.locator(xpath_option).click()

Find the search bar and input the search

In [108]:
#xpath_searchbar = '//*[@id="searchbar"]' 
xpath_searchbar = '//input[@aria-label="Search trademarks"]'

In [109]:
search = page.locator(xpath_searchbar)

In [110]:
# Change this if you like
company = 'NVIDIA'

In [111]:
await search.fill(company)

Make the search

In [112]:
await page.keyboard.press("Enter")

Filter to only live contracts
<br>In other words, filter out the dead ones.

In [116]:
xpath_filter = '//input[@id="statusDead"]'

In [117]:
dead_button = page.locator(xpath_filter)

In [118]:
# click it
await dead_button.click()

# Save results

In [119]:
# how to save what the emulator sees
source = await page.content()
with open(f'data/trademarks_{company}.html', 'w') as f:
    f.write(source)

GO TO THE NEXT PAGE AND REPEAT

In [120]:
# this is the next page button
xpath_next = '//li[@class="page-item"]//i[text() = "navigate_next"]'

In [121]:
# What is the xpath for a result, make sure this is visible before going to next page.
xpath_result = '//div[@id="resultsRow"]'

In [122]:
next_button = page.locator(xpath_next)

In [123]:
await expect(next_button).to_be_visible()

In [124]:
await next_button.click()

In [125]:
await page.locator(xpath_result).is_visible()

True

In [127]:
# Here we'll put it all together and iterate through.
# There are many ways to do this
i = 1
collect = True
while collect:
    # check the next button is visible
    if await page.is_visible(xpath_next):
        # click the next button
        next_button = page.locator(xpath_next)
        await next_button.click()

        # make sure the search result is visible
        await page.locator(xpath_result).is_visible()

        # save the contents
        source = await page.content()
        with open(f'data/trademarks_{company}_{i}.html', 'w') as f:
            f.write(source)
        i += 1
    else:
        collect = False
        break

## Parse the contents

In [128]:
import glob
import pandas as pd
from lxml import etree, html

In [129]:
# list the scraped pages to pars them
files = glob.glob('data/trademarks*.html')

In [130]:
# A trademark entry'
xpath_trademark_cards = './/div[@class="card m-2 result-card ng-star-inserted"]'
# the serial number
xpath_serial = './/div[@class="row mb-2 ng-star-inserted"]//span'

In [133]:
data = []
for fn in files:
    # read into a XML tree from a string
    tree = html.fromstring(open(fn).read())
    trademarks = tree.findall(xpath_trademark_cards)
    for trademark in trademarks:
        serial = trademark.find(xpath_serial)
        img = trademark.find('.//img')
        img_url = None
        if img is not None:
            img_url = img.get('src',)
        row = {'serial' : serial.text, 'img_url': img_url}
        data.append(row)

In [134]:
df = pd.DataFrame(data)
df.head(5)

,serial,img_url
0,88063207,https://tmcms-docs.uspto.gov/cases/88063207/ma...
1,88208710,https://tmcms-docs.uspto.gov/cases/88208710/ma...
2,78419677,https://tmcms-docs.uspto.gov/cases/78419677/ma...
3,85935646,https://tmcms-docs.uspto.gov/cases/85935646/ma...
4,78646210,https://tmcms-docs.uspto.gov/cases/78646210/ma...


In [135]:
df.to_csv('data/trademarks.csv', index=False)